## Grab production results

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
from pandas import ExcelWriter

In [2]:
# Requisite input

economies = ['01_AUS', '02_BD', '03_CDA', '04_CHL', '05_PRC', '06_HKC',
             '07_INA', '08_JPN', '09_ROK', '10_MAS', '11_MEX', '12_NZ',
             '13_PNG', '14_PE', '15_RP', '16_RUS', '17_SIN', '18_CT', '19_THA',
             '20_USA', '21_VN']

In [ ]:
# Read data from projection results

production_data = pd.DataFrame()

for economy in economies:
    pd.read